In [1]:
import datetime as dt
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import tensorflow as tf
import os


1.3.3
2.6.0


In [2]:
data_path = os.getenv('HOME')+'/aiffel/yoochoose/data/'
train_path = data_path+'/ratings.dat'

def load_data(data_path, nrows=None):
    data = pd.read_csv(data_path, sep='::', header=None, usecols=[0, 1, 2, 3], dtype={0: np.int32, 1: np.int32, 2: np.int32}, nrows=nrows)
    data.columns = ['UserId', 'ItemId', 'Rating', 'Time']
    return data

data = load_data(train_path, None)
data.sort_values(['UserId', 'Time'], inplace=True)  # data를 id와 시간 순서로 정렬해줍니다.
data

,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
...,...,...,...,...
1000019,6040,2917,4,997454429
999988,6040,1921,4,997454464
1000172,6040,1784,3,997454464
1000167,6040,161,3,997454486


In [3]:
user_1 = data.loc[data.UserId==1]

print(len(user_1))
print(user_1['ItemId'].nunique())
display(user_1)

53
53


,UserId,ItemId,Rating,Time
31,1,3186,4,978300019
22,1,1270,5,978300055
27,1,1721,4,978300055
37,1,1022,5,978300055
24,1,2340,3,978300103
36,1,1836,5,978300172
3,1,3408,4,978300275
7,1,2804,5,978300719
47,1,1207,4,978300719
0,1,1193,5,978300760


해당 프로젝트는 유저 클릭 심리를 파악하는게 주 목적이다

일단 itemid는 유저가 클릭을 한 아이템을 말한다
<br/>
time은 그 아이템을 언제 클릭했냐는 뜻이고,
<br/>
rating은 그 클릭에 대한 만족도다

확인 결과 모든 값이 유니크하다
<br/>
뭔가 이상하다

UserId를 3까지 확인했는데, 다 유니크했다
<br/>
본문을 확인해보니, 중복 클릭수가 없다고 한다
<br/>
그럼 어떻게 영화를 추천하지??

rating을 클릭수 대신 사용하자
<br/>
일단 유저마다 best 클릭수를 추출한다

프로세스를 알 거 같다
<br/>
먼저 영화 전체 데이터가 담겨져있는 장을 만든다
<br/>
특정 유저 a가 영화를 보고 좋은 평점을 남기면 기록을 남긴다
<br/>
해당 유저 b와 비슷한 유저에게 영화를 추천한다

유저와의 유사성을 어떻게 알지?
<br/>
a와 b의 영화 평점 유사도 구한다
<br/>
유사도가 특정 기준을 넘기면 a와 b가 유사하다 판단한다

유사하다 하더라도 a와 b가 본 영화는 차이가 난다
<br/>
즉 a가 본 영화를 b는 안 봤을 수 있다
<br/>
때문에 서로 영화 기록을 공유하며 영화 기록을 넓혀간다
<br/>
한계점이라 한다면, a와 b가 기록을 전부 공유했다면, 더이상 기록을 넓혀갈 수 없다

즉 워드2임베딩을 할 줄 알아야 된다
<br/>
저번에 워드투임베딩 모델 불러와서 한 걸로 기억하는데,
<br/>
어떻게 하냐...

근데 이렇게 하면 생기는 문제가, 모든 영화가 개별적으로 존재하므로 컴퓨팅 자원을 뒤지게 잡아먹는다
<br/>
이를 해결하기 위해서 영화도 비슷한 것들끼리 묶어주는 워드투임베딩을 했다

영화 유사도를 어떻게 구하느냐
<br/>
일단 모든 영화는 독립적이다
<br/>
제목이 같다 하더라도 장르가 다르면 완전히 다른 작품이다
<br/>
장르가 같다 하더라도 제목이 다르고 배우도 다르고, 감독마저도 다르다면 완전 다른 영화다
<br/>
이걸 어떻게 해결할 것인가?

깃허브에서 찾아봤는데, 없다;
<br/> 대부분이 if문과 for문을 써서 비슷한 단어들끼리 묶어주는 작업을 해주는데,
<br/> 이건 아닌거 같다

https://wikidocs.net/22660
<br/> 위 블로그는 워드투벡터를 서비스하는 곳을 어떻게 쓰는지 알려주는 곳이다
<br/> 추후 이걸 참고해보자

In [4]:
import datetime

def convert_seconds_to_time(in_seconds):
    """초를 입력받아 n days, nn:nn:nn으로 변환"""
    return str(datetime.timedelta(seconds=in_seconds))

출처

https://m.blog.naver.com/wideeyed/221522740612


In [5]:
convert_seconds_to_time(int(user_1['Time'][0]))

for i in user_1['Time']:
    k = convert_seconds_to_time(int(i))
    print(k, i)

11322 days, 22:00:19 978300019
11322 days, 22:00:55 978300055
11322 days, 22:00:55 978300055
11322 days, 22:00:55 978300055
11322 days, 22:01:43 978300103
11322 days, 22:02:52 978300172
11322 days, 22:04:35 978300275
11322 days, 22:11:59 978300719
11322 days, 22:11:59 978300719
11322 days, 22:12:40 978300760
11322 days, 22:12:40 978300760
11322 days, 22:12:40 978300760
11322 days, 22:22:48 978301368
11322 days, 22:23:18 978301398
11322 days, 22:26:10 978301570
11322 days, 22:26:30 978301590
11322 days, 22:26:59 978301619
11322 days, 22:28:33 978301713
11322 days, 22:29:12 978301752
11322 days, 22:29:13 978301753
11322 days, 22:29:13 978301753
11322 days, 22:29:37 978301777
11322 days, 22:29:37 978301777
11322 days, 22:29:37 978301777
11322 days, 22:32:33 978301953
11322 days, 22:32:48 978301968
11322 days, 22:33:59 978302039
11322 days, 22:33:59 978302039
11322 days, 22:34:51 978302091
11322 days, 22:34:51 978302091
11322 days, 22:35:09 978302109
11322 days, 22:35:24 978302124
11322 da

time 데이터가 어떤 단위인지 알 수 없었지만,
<br/> 초단위라고 가정하고 계산해본 결과 86400 초가 지나면 24시간이 지난다는 사실을 알게 되었다
<br/> 그럼 Time 데이터 앞뒤가 86400 이상으로 차이나면,
<br/> 혹은 사이트 평균 유지 시간을 구해서 그 값 이상으로 차이나면 새로운 session이라고 판단한다
<br/> 이것도 정확도가 낮다면, 사이트 세션 유지 시간 이상으로 차이가 난다면, 새로운 session이라 판단한다

근데 굳이 session id를 알아낼 필요가 있나??

다른 방법으로는 userid와 itemid

In [6]:
print(len(data))
print(len(user_1))
user_1.ItemId.nunique()

1000209
53


53

In [7]:
a_count = 0

def user_session(data, id, ratio=86400/4):

    user_1 = data.loc[data.UserId==id]

    prev = 0
    count = 0
    term = []
    
    
    for i in user_1['Time']:
        
        if prev == 0:
            prev = i
            term.append([count, prev])
    
        elif abs(prev-i)<ratio:
    
            term.append([count, i])
    
        elif abs(prev-i)>ratio:
            count +=1
            prev=i
            term.append([count, i])
    
    return term
        

a_count = 0

def user_session(data, id, ratio=86400/4):

    user_1 = data.loc[data.UserId==id]

    prev = 0
    count = 0
    term = []
    
    
    for i in user_1['Time']:
        
        if prev == 0:
            prev = i
            term.append([])
            term[count].append([count, prev])
        
        elif abs(prev-i)<ratio:
            term[count].append([count, i])
    
        elif abs(prev-i)>ratio:
            term.append(list())
            count +=1
            prev=i
            term[count].append([count, i])
        
    return term
        

In [8]:
data.UserId.nunique()

6040

In [9]:

def user_session(data, ratio=86400/4):
    term = []
    
    count = 0
    for k in list(range(1,data.UserId.nunique()+1)):
        user_1 = data.loc[data.UserId==k]

        prev = 0
    
        for i in user_1['Time']:
        
            if prev == 0:
                prev = i
                term.append(count)
    
            elif abs(prev-i)<ratio:
    
                term.append(count)
    
            elif abs(prev-i)>ratio:
                count +=1
                prev=i
                term.append(count)
            
        count+=1
    return term
        

In [10]:
user_session = user_session(data)

print(len(user_session))

1000207


In [11]:
len(user_session)

1000207

In [12]:
user_session[-1:]

[23154]

In [13]:
user_session.append(23154)
user_session.append(23155)

In [14]:
data['session'] = user_session

In [21]:
data.index = list(range(len(data)))

텀이 86400/2보다 크면, 카운터를 나누게 만들었다
<br/> 이제 이걸 전체 데이터에 적용해야 된다


In [22]:
data

,UserId,ItemId,Rating,Time,session
0,1,3186,4,978300019,0
1,1,1270,5,978300055,0
2,1,1721,4,978300055,0
3,1,1022,5,978300055,0
4,1,2340,3,978300103,0
...,...,...,...,...,...
1000204,6040,2917,4,997454429,23153
1000205,6040,1921,4,997454464,23153
1000206,6040,1784,3,997454464,23154
1000207,6040,161,3,997454486,23154


In [23]:
oldest, latest = data['Time'].min(), data['Time'].max()
print(oldest) 
print(latest)

956703932
1046454590


In [24]:
years_ago = latest - (86400*365*3)   # 최종 날짜로부터 3년 이전 날짜를 구한다.  
data = data[data['Time'] > years_ago]   # 방금 구한 날짜 이후의 데이터만 모은다. 
data

,UserId,ItemId,Rating,Time,session
0,1,3186,4,978300019,0
1,1,1270,5,978300055,0
2,1,1721,4,978300055,0
3,1,1022,5,978300055,0
4,1,2340,3,978300103,0
...,...,...,...,...,...
1000204,6040,2917,4,997454429,23153
1000205,6040,1921,4,997454464,23153
1000206,6040,1784,3,997454464,23154
1000207,6040,161,3,997454486,23154


In [27]:
# 날짜를 기준으로 분리
def split_by_date(data: pd.DataFrame, n_days: int):
    final_time = data['Time'].max()
    session_last_time = data.groupby('session')['Time'].max()
    session_in_train = session_last_time[session_last_time < final_time - (86400*n_days)].index
    session_in_test = session_last_time[session_last_time >= final_time - (86400*n_days)].index

    before_date = data[data['session'].isin(session_in_train)]
    after_date = data[data['session'].isin(session_in_test)]
    after_date = after_date[after_date['ItemId'].isin(before_date['ItemId'])]
    return before_date, after_date

In [28]:
tr, test = split_by_date(data, n_days=100)
tr, val = split_by_date(tr, n_days=100)

In [29]:
# data에 대한 정보를 살펴봅니다.
def stats_info(data: pd.DataFrame, status: str):
    first_time = dt.datetime.utcfromtimestamp(data["Time"].min()).strftime('%Y-%m-%d %H:%M:%S')
    last_time = dt.datetime.utcfromtimestamp(data["Time"].max()).strftime('%Y-%m-%d %H:%M:%S')
    print(f'* {status} Set Stats Info\n'
          f'\t Events: {len(data)}\n'
          f'\t Sessions: {data["UserId"].nunique()}\n'
          f'\t Items: {data["ItemId"].nunique()}\n'
          f'\t First Time : {first_time}\n'
          f'\t Last Time : {last_time}\n')

In [30]:
stats_info(tr, 'train')
stats_info(val, 'valid')
stats_info(test, 'test')

* train Set Stats Info
	 Events: 966950
	 Sessions: 6039
	 Items: 3697
	 First Time : 2000-04-25 23:05:32
	 Last Time : 2002-08-12 13:53:29

* valid Set Stats Info
	 Events: 12266
	 Sessions: 300
	 Items: 2494
	 First Time : 2000-04-27 15:17:08
	 Last Time : 2002-11-20 16:38:40

* test Set Stats Info
	 Events: 20984
	 Sessions: 420
	 Items: 2659
	 First Time : 2000-05-01 05:44:56
	 Last Time : 2003-02-28 17:49:50



In [31]:
# train set에 없는 아이템이 val, test기간에 생길 수 있으므로 train data를 기준으로 인덱싱합니다.
id2idx = {item_id : index for index, item_id in enumerate(tr['ItemId'].unique())}

def indexing(df, id2idx):
    df['item_idx'] = df['ItemId'].map(lambda x: id2idx.get(x, -1))  # id2idx에 없는 아이템은 모르는 값(-1) 처리 해줍니다.
    return df

tr = indexing(tr, id2idx)
val = indexing(val, id2idx)
test = indexing(test, id2idx)

In [34]:

data_path = Path(os.getenv('HOME')+'/aiffel/yoochoose/data')
train_path = data_path / 'ratings.dat'

In [35]:
# 전처리가 완료된 데이터를 저장합니다.
save_path = data_path / 'processed'
save_path.mkdir(parents=True, exist_ok=True)

tr.to_pickle(save_path / 'train.pkl')
val.to_pickle(save_path / 'valid.pkl')
test.to_pickle(save_path / 'test.pkl')

In [40]:
class SessionDataset:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, data):
        self.df = data
        self.click_offsets = self.get_click_offsets()
        self.session_idx = np.arange(self.df['session'].nunique())  # indexing to SessionId

    def get_click_offsets(self):
        """
        Return the indexes of the first click of each session IDs,
        """
        offsets = np.zeros(self.df['session'].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby('session').size().cumsum()
        return offsets

In [41]:
tr_dataset = SessionDataset(tr)
tr_dataset.df.head(10)

,UserId,ItemId,Rating,Time,session,item_idx
0,1,3186,4,978300019,0,0
1,1,1270,5,978300055,0,1
2,1,1721,4,978300055,0,2
3,1,1022,5,978300055,0,3
4,1,2340,3,978300103,0,4
5,1,1836,5,978300172,0,5
6,1,3408,4,978300275,0,6
7,1,2804,5,978300719,0,7
8,1,1207,4,978300719,0,8
9,1,1193,5,978300760,0,9


In [42]:
tr_dataset.click_offsets
tr_dataset.session_idx

array([    0,     1,     2, ..., 21592, 21593, 21594])

In [43]:
class SessionDataLoader:
    """Credit to yhs-968/pyGRU4REC."""

    def __init__(self, dataset: SessionDataset, batch_size=50):
        self.dataset = dataset
        self.batch_size = batch_size

    def __iter__(self):
        """ Returns the iterator for producing session-parallel training mini-batches.
        Yields:
            input (B,):  Item indices that will be encoded as one-hot vectors later.
            target (B,): a Variable that stores the target item indices
            masks: Numpy array indicating the positions of the sessions to be terminated
        """

        start, end, mask, last_session, finished = self.initialize()  # initialize 메소드에서 확인해주세요.
        """
        start : Index Where Session Start
        end : Index Where Session End
        mask : indicator for the sessions to be terminated
        """

        while not finished:
            min_len = (end - start).min() - 1  # Shortest Length Among Sessions
            for i in range(min_len):
                # Build inputs & targets
                inp = self.dataset.df['item_idx'].values[start + i]
                target = self.dataset.df['item_idx'].values[start + i + 1]
                yield inp, target, mask

            start, end, mask, last_session, finished = self.update_status(start, end, min_len, last_session, finished)

    def initialize(self):
        first_iters = np.arange(self.batch_size)    # 첫 배치에 사용할 세션 Index를 가져옵니다.
        last_session = self.batch_size - 1    # 마지막으로 다루고 있는 세션 Index를 저장해둡니다.
        start = self.dataset.click_offsets[self.dataset.session_idx[first_iters]]       # data 상에서 session이 시작된 위치를 가져옵니다.
        end = self.dataset.click_offsets[self.dataset.session_idx[first_iters] + 1]  # session이 끝난 위치 바로 다음 위치를 가져옵니다.
        mask = np.array([])   # session의 모든 아이템을 다 돌은 경우 mask에 추가해줄 것입니다.
        finished = False         # data를 전부 돌았는지 기록하기 위한 변수입니다.
        return start, end, mask, last_session, finished

    def update_status(self, start: np.ndarray, end: np.ndarray, min_len: int, last_session: int, finished: bool):  
        # 다음 배치 데이터를 생성하기 위해 상태를 update합니다.
        
        start += min_len   # __iter__에서 min_len 만큼 for문을 돌았으므로 start를 min_len 만큼 더해줍니다.
        mask = np.arange(self.batch_size)[(end - start) == 1]  
        # end는 다음 세션이 시작되는 위치인데 start와 한 칸 차이난다는 것은 session이 끝났다는 뜻입니다. mask에 기록해줍니다.

        for i, idx in enumerate(mask, start=1):  # mask에 추가된 세션 개수만큼 새로운 세션을 돌것입니다.
            new_session = last_session + i  
            if new_session > self.dataset.session_idx[-1]:  # 만약 새로운 세션이 마지막 세션 index보다 크다면 모든 학습데이터를 돈 것입니다.
                finished = True
                break
            # update the next starting/ending point
            start[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session]]     # 종료된 세션 대신 새로운 세션의 시작점을 기록합니다.
            end[idx] = self.dataset.click_offsets[self.dataset.session_idx[new_session] + 1]

        last_session += len(mask)  # 마지막 세션의 위치를 기록해둡니다.
        return start, end, mask, last_session, finished
    

In [44]:
tr_data_loader = SessionDataLoader(tr_dataset, batch_size=4)
tr_dataset.df.head(15)

,UserId,ItemId,Rating,Time,session,item_idx
0,1,3186,4,978300019,0,0
1,1,1270,5,978300055,0,1
2,1,1721,4,978300055,0,2
3,1,1022,5,978300055,0,3
4,1,2340,3,978300103,0,4
5,1,1836,5,978300172,0,5
6,1,3408,4,978300275,0,6
7,1,2804,5,978300719,0,7
8,1,1207,4,978300719,0,8
9,1,1193,5,978300760,0,9


In [45]:
iter_ex = iter(tr_data_loader)

In [46]:
inputs, labels, mask =  next(iter_ex)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

Model Input Item Idx are : [ 0 40 53 65]
Label Item Idx are :       [ 1 41 54 62]
Previous Masked Input Idx are []


In [47]:
def mrr_k(pred, truth: int, k: int):
    indexing = np.where(pred[:k] == truth)[0]
    if len(indexing) > 0:
        return 1 / (indexing[0] + 1)
    else:
        return 0

def recall_k(pred, truth: int, k: int) -> int:
    answer = truth in pred[:k]
    return int(answer)

In [48]:
from tensorflow.keras.layers import Input, Dense, Dropout, GRU
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [52]:
def create_model(args):
    inputs = Input(batch_shape=(args.batch_size, 1, args.num_items))
    gru, _ = GRU(args.hsz, stateful=True, return_state=True, name='GRU')(inputs)
    dropout = Dropout(args.drop_rate)(gru)
    predictions = Dense(args.num_items, activation='softmax')(dropout)
    model = Model(inputs=inputs, outputs=[predictions])
    model.compile(loss=categorical_crossentropy, optimizer=Adam(args.lr), metrics=['accuracy'])
    model.summary()
    return model

In [53]:
class Args:
    def __init__(self, tr, val, test, batch_size, hsz, drop_rate, lr, epochs, k):
        self.tr = tr
        self.val = val
        self.test = test
        self.num_items = tr['ItemId'].nunique()
        self.num_sessions = tr['session'].nunique()
        self.batch_size = batch_size
        self.hsz = hsz
        self.drop_rate = drop_rate
        self.lr = lr
        self.epochs = epochs
        self.k = k

args = Args(tr, val, test, batch_size=128, hsz=50, drop_rate=0.1, lr=0.001, epochs=10, k=20)

In [54]:
model = create_model(args)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(128, 1, 3697)]          0         
_________________________________________________________________
GRU (GRU)                    [(128, 50), (128, 50)]    562350    
_________________________________________________________________
dropout (Dropout)            (128, 50)                 0         
_________________________________________________________________
dense (Dense)                (128, 3697)               188547    
Total params: 750,897
Trainable params: 750,897
Non-trainable params: 0
_________________________________________________________________


In [57]:
# train 셋으로 학습하면서 valid 셋으로 검증합니다.
def train_model(model, args):
    train_dataset = SessionDataset(args.tr)
    train_loader = SessionDataLoader(train_dataset, batch_size=args.batch_size)

    for epoch in range(1, args.epochs + 1):
        total_step = len(args.tr) - args.tr['session'].nunique()
        tr_loader = tqdm(train_loader, total=total_step // args.batch_size, desc='Train', mininterval=1)
        for feat, target, mask in tr_loader:
            reset_hidden_states(model, mask)  # 종료된 session은 hidden_state를 초기화합니다. 아래 메서드에서 확인해주세요.

            input_ohe = to_categorical(feat, num_classes=args.num_items)
            input_ohe = np.expand_dims(input_ohe, axis=1)
            target_ohe = to_categorical(target, num_classes=args.num_items)

            result = model.train_on_batch(input_ohe, target_ohe)
            tr_loader.set_postfix(train_loss=result[0], accuracy = result[1])

        val_recall, val_mrr = get_metrics(args.val, model, args, args.k)  # valid set에 대해 검증합니다.

        print(f"\t - Recall@{args.k} epoch {epoch}: {val_recall:3f}")
        print(f"\t - MRR@{args.k}    epoch {epoch}: {val_mrr:3f}\n")


def reset_hidden_states(model, mask):
    gru_layer = model.get_layer(name='GRU')  # model에서 gru layer를 가져옵니다.
    hidden_states = gru_layer.states[0].numpy()  # gru_layer의 parameter를 가져옵니다.
    for elt in mask:  # mask된 인덱스 즉, 종료된 세션의 인덱스를 돌면서
        hidden_states[elt, :] = 0  # parameter를 초기화 합니다.
    gru_layer.reset_states(states=hidden_states)


def get_metrics(data, model, args, k: int):  # valid셋과 test셋을 평가하는 코드입니다. 
                                             # train과 거의 같지만 mrr, recall을 구하는 라인이 있습니다.
    dataset = SessionDataset(data)
    loader = SessionDataLoader(dataset, batch_size=args.batch_size)
    recall_list, mrr_list = [], []

    total_step = len(data) - data['session'].nunique()
    for inputs, label, mask in tqdm(loader, total=total_step // args.batch_size, desc='Evaluation', mininterval=1):
        reset_hidden_states(model, mask)
        input_ohe = to_categorical(inputs, num_classes=args.num_items)
        input_ohe = np.expand_dims(input_ohe, axis=1)

        pred = model.predict(input_ohe, batch_size=args.batch_size)
        pred_arg = tf.argsort(pred, direction='DESCENDING')  # softmax 값이 큰 순서대로 sorting 합니다.

        length = len(inputs)
        recall_list.extend([recall_k(pred_arg[i], label[i], k) for i in range(length)])
        mrr_list.extend([mrr_k(pred_arg[i], label[i], k) for i in range(length)])

    recall, mrr = np.mean(recall_list), np.mean(mrr_list)
    return recall, mrr

In [ ]:
train_model(model, args)

Evaluation:  26%|██▌       | 23/89 [00:15<00:43,  1.50it/s]


	 - Recall@20 epoch 1: 0.129755
	 - MRR@20    epoch 1: 0.031217



Evaluation:  26%|██▌       | 23/89 [00:14<00:41,  1.58it/s]


	 - Recall@20 epoch 2: 0.155571
	 - MRR@20    epoch 2: 0.043096



Evaluation:  26%|██▌       | 23/89 [00:14<00:40,  1.61it/s]


...

Evaluation:  26%|██▌       | 23/89 [00:14<00:41,  1.58it/s]


	 - Recall@20 epoch 4: 0.166780
	 - MRR@20    epoch 4: 0.054811



Evaluation:  26%|██▌       | 23/89 [00:14<00:41,  1.61it/s]


	 - Recall@20 epoch 5: 0.171875
	 - MRR@20    epoch 5: 0.057144



Train:  35%|███▌      | 2614/7385 [01:39<03:05, 25.78it/s, accuracy=0.0469, train_loss=5.51] 

In [ ]:
def test_model(model, args, test):
    test_recall, test_mrr = get_metrics(test, model, args, 20)
    print(f"\t - Recall@{args.k}: {test_recall:3f}")
    print(f"\t - MRR@{args.k}: {test_mrr:3f}\n")

test_model(model, args, test)

# 회고

이번에 프로젝트를 진행하면서 가장 힘들었던게 바로 class 해부와 데이터프레임 편집이었습니다
<br/> 먼저 class 해부

특정 목적을 가지고 임의로 만든 class를 해석하는 건 어렵지 않지만,
<br/> class 인스턴스가 엉켜있으면 스크롤을 아주 많이 움직여서 class를 다시 확인해야 합니다
<br/> 이게 힘든 이유가 만약 데이터 구조를 왜 이렇게 쌓았지? 라는 의문이 생기면,
<br/> 혹은 class 인자를 바꿔줘야만 된다면,
<br/> 임의로 제작된 class로 다시 올라가서 편집해줘야 됩니다
<br/> 근데 여기서 또 문제가 뭐냐,
<br/> 코드를 다 해석할 수 있어도 파이썬 코드를 처음부터 쌓다보면 번아웃인지 뭔지는 모르겠지만
<br/> 원래 잘 쓰던 for문, if문, 파일 불러오는 법 등을 까먹게 됩니다

뭐 덕분에 코드는 최대한 내가 손대는 부분을 적게 조절하자는 교훈을 얻었지만요

두번째로 데이터프레임 편집
<br/> 데이터톤 할 때도 느꼈지만, 제 데이터프레임 편집 능력을 오만했나 봅니다
<br/> 데이터가 크면 클수록, 데이터의 예상치 못한 변수가 많아집니다
<br/> session을 만들 때 Time 데이터 전과 현재의 차이가 12시간이 지나면, session id가 새로 생겨날 거라고 생각해서 전처리 했는데,
<br/> 1000207
<br/> 데이터프레임 길이보다 2가 작은 값이출력되었습니다
<br/> 이걸 맞추려면 처음부터 끝까지 데이터를 확인해서 어디가 잘못되었는지 찾아야 되는데,
<br/> 그럴 시간도 없고, 그렇게 문제를 해결한다면 코딩을 할 필요가 없다 생각이 들어
<br/> 그냥 맨 마지막 값만 확인하고 임의로 추가해줬습니다

나중에 다시 볼 영상
<br/> :
<br/> https://youtu.be/6TP51jvjLsE
<br/> https://youtu.be/mLwMe4KUZz8